In [ ]:
!pip install gosling
import gosling as gos

# Datasets

In [ ]:
mv = gos.multivec(
    url="https://server.gosling-lang.org/api/v1/tileset_info/?d=cistrome-multivec",
    row="sample",
    column="position",
    value="peak",
    categories=["sample 1", "sample 2", "sample 3", "sample 4"],
    binSize=2
)

sv = gos.csv(
    url="https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/rearrangements.bulk.1639.simple.filtered.pub",
    headerNames=["chr1", "p1s", "p1e", "chr2", "p2s", "p2e", "type", "id", "f1", "f2", "f3", "f4", "f5", "f6"],
    separator="\t",
    genomicFieldsToConvert=[
        {"chromosomeField": "chr1", "genomicFields": ["p1s", "p1e"]},
        {"chromosomeField": "chr2", "genomicFields": ["p2s", "p2e"]}
    ]
)

# Individual Tracks & Views

In [ ]:
bar_base = gos.Track(data=mv).mark_bar().encode(
    x='start:G',
    xe='end:G',
    y='peak:Q',
    color=gos.Color('sample:N', range=['steelblue', 'salmon', 'steelblue', 'salmon']),
    stroke=gos.value('white'),
    strokeWidth=gos.value(2),
)

bar_overview = gos.overlay(
    bar_base,
    bar_base.mark_brush().encode(x=gos.X(linkingId='detail-1')),
    bar_base.mark_brush().encode(x=gos.X(linkingId='detail-2'))
).properties(width=300, height=300)

link = gos.Track(data=sv).mark_withinLink().encode(
    x='p1s:G',
    xe='p2e:G',
    stroke=gos.Stroke('chr1:N', range=['steelblue']),
    strokeWidth=gos.value(2),
    opacity=gos.value(0.6)
).properties(width=300, height=100)

overview = gos.stack(bar_overview, link).properties(layout="circular", spacing=1)

overview

In [ ]:
detail_1 = bar_base.encode(strokeWidth=gos.value(0)).properties(width=300, height=100).view(
    linkingId='detail-1',
    xDomain=gos.GenomicDomain(chromosome='3'),
)

detail_2 = bar_base.encode(strokeWidth=gos.value(0)).properties(width=300, height=100).view(
    linkingId='detail-2',
    xDomain=gos.GenomicDomain(chromosome='10'),
)

detail_view = gos.vertical(detail_1, detail_2).properties(spacing=20)
detail_view

# Compose Views

In [ ]:
gos.horizontal(overview, detail_view).properties(
    title='Overview and Detail Views',
)